In [1]:
import os
import sys
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import seaborn as sns
from torch.distributions import Normal
from src.strategy.buffer import Buffer
%matplotlib inline
sns.set_theme()

In [2]:
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)

In [3]:
from src.position_sizing import fiducia_calculator
from src.strategy.model import Model
from src.utils import get_config, read_file
config = get_config.read_yaml()

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
input_dim = len(config['data']['symbols']) * config['data']['num_features']
gamma = config['hyperparameters']['gamma']
gae_lambda = config['hyperparameters']['gae_lambda']
clip_epsilon = config['hyperparameters']['clip_epsilon']
value_loss_coef = config['hyperparameters']['value_loss_coef']
entropy_loss_coef = config['hyperparameters']['entropy_loss_coef']
batch_size = config['hyperparameters']['batch_size']
epochs = config['hyperparameters']['num_epochs']
lr = config['hyperparameters']['learning_rate']
window_size = config['hyperparameters']['window_size']
print(f'input_dim: {input_dim}')
print(f'gamma: {gamma}')
print(f'gae_lambda: {gae_lambda}')
print(f'clip_epsilon: {clip_epsilon}')
print(f'value_loss_coef: {value_loss_coef}')
print(f'entropy_loss_coef: {entropy_loss_coef}')
print(f'batch_size: {batch_size}')
print(f'epochs: {epochs}')
print(f'lr: {lr}')
print(f'window_size: {window_size}')
print(f'device: {device}')

input_dim: 126
gamma: 0.99
gae_lambda: 0.95
clip_epsilon: 0.2
value_loss_coef: 0.5
entropy_loss_coef: 0.01
batch_size: 128
epochs: 50
lr: 0.001
window_size: 72
device: cuda


In [5]:
data = read_file.read_merged_training_data()
data

,"('open', 'ETH')","('high', 'ETH')","('low', 'ETH')","('close', 'ETH')","('volume', 'ETH')","('rsi', 'ETH')","('sma-50', 'ETH')","('sma-100', 'ETH')","('sma-200', 'ETH')","('ema-50', 'ETH')",...,"('volume', 'XLM')","('rsi', 'XLM')","('sma-50', 'XLM')","('sma-100', 'XLM')","('sma-200', 'XLM')","('ema-50', 'XLM')","('ema-100', 'XLM')","('ema-200', 'XLM')","('atr', 'XLM')","('adx', 'XLM')"
timestamp,,,,,,,,,,,,,,,,,,,,,
2020-10-01 00:00:00,1.000000,0.962171,0.986505,0.930535,0.618368,0.904891,0.000000,1.0,1.0,0.887498,...,0.215989,1.000000,0.004091,0.141368,0.999997,0.466309,0.580954,0.298796,0.257085,0.325054
2020-10-01 01:00:00,0.922741,1.000000,0.971660,1.000000,0.374792,1.000000,0.073192,1.0,1.0,1.000000,...,0.002807,1.000000,0.061674,0.203590,0.999997,0.638343,0.796008,0.415248,0.064341,0.406695
2020-10-01 02:00:00,1.000000,1.000000,1.000000,1.000000,0.968396,1.000000,0.151500,1.0,1.0,1.000000,...,0.647335,1.000000,0.121460,0.268277,0.999998,0.869845,0.999995,0.577327,0.249065,0.519852
2020-10-01 03:00:00,1.000000,0.932530,1.000000,0.971104,0.200648,0.953461,0.235908,1.0,1.0,1.000000,...,0.615060,0.576580,0.153241,0.303039,0.999998,0.982428,0.999995,0.659771,0.499462,0.568566
2020-10-01 04:00:00,0.970954,1.000000,1.000000,0.969040,0.453046,0.949955,0.318741,1.0,1.0,1.000000,...,0.200664,0.534733,0.182190,0.325396,0.999998,0.999998,0.999996,0.722614,0.372141,0.685574
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-09-29 19:00:00,0.616359,0.535242,0.735184,0.694987,0.052053,1.000000,0.664608,1.0,1.0,0.057490,...,0.031134,0.724355,0.999999,0.999999,0.999998,0.999999,0.999999,0.999999,0.637189,1.000000
2024-09-29 20:00:00,0.694987,0.694383,0.623045,0.539578,0.232078,0.798289,0.554950,1.0,1.0,0.042796,...,0.119612,0.470556,0.999999,0.999999,0.999998,0.999999,0.999999,0.999999,0.717075,0.990767
2024-09-29 21:00:00,0.539842,0.566630,0.669531,0.691029,0.062073,0.985836,0.428238,1.0,1.0,0.065734,...,0.041038,0.515497,0.999999,0.999999,0.999998,0.999999,0.999999,0.999999,0.609842,0.982194


In [6]:
model = Model(input_dim).to(device)
model

Model(
  (lstm): LSTM(126, 256, batch_first=True)
  (actor_head): Sequential(
    (0): Linear(in_features=256, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=20, bias=True)
  )
  (critic_head): Sequential(
    (0): Linear(in_features=256, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=1, bias=True)
  )
)

In [7]:
h_0, c_0 = model.init_hidden_state(batch_size, device)
h_0.shape

torch.Size([1, 128, 256])

In [8]:
x = []
for _ in range(batch_size):
    y = []
    for i in range(window_size):
        y.append(data.loc[data.index[i + _]])
    x.append(y)
x_t = torch.tensor(x).float().to(device)
print(x_t)
print(x_t.shape)

C:\Users\Akhtar_new\AppData\Local\Temp\ipykernel_21800\703557019.py:7: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  x_t = torch.tensor(x).float().to(device)


tensor([[[1.0000, 0.9622, 0.9865,  ..., 0.2988, 0.2571, 0.3251],
         [0.9227, 1.0000, 0.9717,  ..., 0.4152, 0.0643, 0.4067],
         [1.0000, 1.0000, 1.0000,  ..., 0.5773, 0.2491, 0.5199],
         ...,
         [0.8516, 0.7520, 0.8283,  ..., 0.0000, 0.0000, 0.0000],
         [0.8287, 0.6779, 0.5724,  ..., 0.0000, 0.0000, 0.0000],
         [0.6659, 0.4928, 0.4977,  ..., 0.0000, 0.0122, 0.0000]],

        [[0.9227, 1.0000, 0.9717,  ..., 0.4152, 0.0643, 0.4067],
         [1.0000, 1.0000, 1.0000,  ..., 0.5773, 0.2491, 0.5199],
         [1.0000, 0.9325, 1.0000,  ..., 0.6598, 0.4995, 0.5686],
         ...,
         [0.8287, 0.6779, 0.5724,  ..., 0.0000, 0.0000, 0.0000],
         [0.6659, 0.4928, 0.4977,  ..., 0.0000, 0.0122, 0.0000],
         [0.4777, 0.3929, 0.4252,  ..., 0.0000, 0.0403, 0.0000]],

        [[1.0000, 1.0000, 1.0000,  ..., 0.5773, 0.2491, 0.5199],
         [1.0000, 0.9325, 1.0000,  ..., 0.6598, 0.4995, 0.5686],
         [0.9710, 1.0000, 1.0000,  ..., 0.7226, 0.3721, 0.

In [10]:
action, log_prob, entropy, value, hidden_state_out, buffer = model.get_action_and_value(x_t, (h_0, c_0))
print(f'action: {action}')
print(f'value: {value}')

action: tensor([[ 1.2800, -0.0814, -1.3604,  ...,  0.0549,  0.8407, -0.5140],
        [-0.3040, -1.3986,  0.6043,  ..., -0.0949, -1.2398, -0.0573],
        [-0.4217, -0.4650, -0.4394,  ..., -0.1909, -0.4343,  0.1644],
        ...,
        [-0.4138, -0.8733,  0.3685,  ...,  0.6664,  0.2440, -1.6865],
        [-0.8714,  1.0290,  1.2869,  ..., -0.9988, -0.1955, -0.3957],
        [-0.5978,  0.4288, -2.1075,  ...,  0.2277,  0.8622, -1.5009]],
       device='cuda:0')
value: tensor([[ 5.8625e-04],
        [ 1.5887e-03],
        [ 3.0021e-03],
        [ 8.2306e-04],
        [ 4.6180e-03],
        [ 1.9724e-03],
        [-6.1993e-04],
        [-8.8410e-03],
        [-7.3584e-03],
        [-5.8592e-03],
        [-1.7949e-03],
        [-1.6314e-03],
        [-2.1277e-03],
        [-5.4715e-03],
        [-7.9472e-03],
        [-3.0689e-03],
        [-7.4873e-04],
        [-2.7209e-03],
        [-5.6851e-03],
        [-1.0651e-02],
        [-1.8581e-02],
        [-1.8883e-02],
        [-1.3906e-02]

In [11]:
fiducae = fiducia_calculator.calculate(action)
fiducae

tensor([[ 0.4197, -0.0753, -0.2706,  ...,  0.1233,  0.2705,  0.1865],
        [-0.1037, -0.3100,  0.1972,  ..., -0.0842, -0.2645,  0.2095],
        [-0.1744, -0.1821, -0.1775,  ..., -0.1384, -0.1766,  0.1172],
        ...,
        [-0.1969, -0.3118,  0.1260,  ...,  0.1697,  0.1112,  0.1033],
        [-0.1909,  0.2195,  0.2841,  ..., -0.2169, -0.0971,  0.1313],
        [-0.1622,  0.1327, -0.7340,  ...,  0.1085,  0.2047,  0.1057]],
       device='cuda:0')